# Pytorch inference ensemble notebook

thanks to https://www.kaggle.com/andretugan/pre-trained-roberta-solution-in-pytorch

This comptition summary.


### preprocess

[make NLP](https://www.kaggle.com/kunihikofurugori/make-nlpdataset)

→ not effective (score is not improved.)

[exclude anomaly data](https://www.kaggle.com/kunihikofurugori/step1-exclude-anomaly)

→ score slightly is improved....?

### transformer training

[transformer tpu8fold](https://www.kaggle.com/kunihikofurugori/tpu8fold-transformer)

→ The score is improved because I tried to many seed pattern.

(Please see Section 4.2 in [Fine-Tuning Pretrained Language Models: Weight Initializations, Data Orders, and Early Stopping](https://arxiv.org/pdf/2002.06305.pdf))


[Add attention and early stopping](https://www.kaggle.com/kunihikofurugori/tpu8fold-transformer-attention)
→ attention and early stopping is powerful method. these method is effective.

(Please see Section 5 in [Fine-Tuning Pretrained Language Models: Weight Initializations, Data Orders, and Early Stopping](https://arxiv.org/pdf/2002.06305.pdf))

[Add multi dropout](https://www.kaggle.com/kunihikofurugori/step2-1-tpu8foldtraining-rbase)

→ score slightly is improved....?

### inference

xgb,lgb inference model is slightly imporved. 

svm inference model is not imporved.

LB score in my model ensembleing is 0.461 and andretugan model is model 0.467 and then 2 pair ensemble is then LB 0.457.

In [ ]:
import torch
import pandas as pd
import numpy as np
#torch
import torch
import torch.nn as nn
from torch.nn import Parameter
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import _LRScheduler
from torch.optim import Adam, lr_scheduler

import xgboost as xgb
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer,AutoModel
from sklearn.model_selection import KFold, StratifiedKFold

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

import warnings
warnings.simplefilter('ignore')

In [ ]:
class testDataset:
    def __init__(self, excerpt, tokenizer, max_len):
        self.excerpt = excerpt
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.excerpt)

    def __getitem__(self, item):
        text = str(self.excerpt[item])
        inputs = self.tokenizer(
            text, 
            max_length=self.max_len, 
            padding="max_length", 
            truncation=True
        )

        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]

        return {
            "input_ids": torch.tensor(ids, dtype=torch.long),
            "attention_mask": torch.tensor(mask, dtype=torch.long),
        }

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        self.middle_features = hidden_dim

        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim
        self._init_params()    
    def _init_params(self):
        nn.init.xavier_normal_(self.W.weight)
        nn.init.constant_(self.W.bias, 0)
        nn.init.xavier_normal_(self.V.weight)
        nn.init.constant_(self.V.bias, 0)

    def forward(self, features):
        att = torch.tanh(self.W(features))

        score = self.V(att)

        attention_weights = torch.softmax(score, dim=1)

        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector

class dModel(nn.Module):
    def __init__(self,modelpath, leng):
        super(dModel, self).__init__()
        self.roberta = transformers.AutoModel.from_pretrained(modelpath)#,num_labels=1)
        self.head = AttentionHead(leng,leng,1)
        self.fc = nn.Linear(leng, 1)
        self.dropout = nn.Dropout(p=0.1)
        self.dropouts = nn.ModuleList([
                nn.Dropout(p=0.1) for _ in range(5)
            ])
        self._init_params()    
    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
    def forward(self, **x):
        x = self.roberta(**x)[0]
        x = self.head(x)
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                logits = self.fc(dropout(x))
            else:
                logits += self.fc(dropout(x))
        
        logits /= len(self.dropouts)
        return logits

In [ ]:
def generate_embeddings(model_path,weight_path, max_len,leng):
    model = dModel(model_path,leng)
    model.load_state_dict(torch.load(weight_path,map_location=device))
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = model.to(device)
    model.eval()
    df = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
    dataset = testDataset(excerpt=df.excerpt.values, tokenizer=tokenizer, max_len=max_len)
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=16, num_workers=2, pin_memory=True, shuffle=False, drop_last=False
    )
    final_output = np.empty((0,leng))

    for b_idx, (data) in enumerate(data_loader):
        with torch.no_grad():
            for key, value in data.items():
                data[key] = value.to(device)
            x = model.roberta(**data)[0]
            output = model.head(x)
            final_output = np.append(final_output,output.clone().detach().cpu().numpy(),axis=0)
    
    torch.cuda.empty_cache()
    return final_output

def generate_predictions_md(model_path,weight_path, max_len,leng):
    #model = AutoModelForSequenceClassification.from_pretrained(model_path,num_labels=1)
    model = dModel(model_path,leng)
    model.load_state_dict(torch.load(weight_path,map_location=device))
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    model = model.to(device)
    model.eval()
    df = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
    dataset = testDataset(excerpt=df.excerpt.values, tokenizer=tokenizer, max_len=max_len)
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=8, num_workers=2, pin_memory=True, shuffle=False, drop_last=False
    )
    final_output = []

    for b_idx, data in enumerate(data_loader):
        with torch.no_grad():
            for key, value in data.items():
                data[key] = value.to(device)
            output = model(**data)
            #output = output.logits.squeeze(-1).detach().cpu().numpy()
            output = output.squeeze(-1).detach().cpu().numpy()
            final_output = np.append(final_output,output)
    
    torch.cuda.empty_cache()
    return final_output

In [ ]:
#0:LB 0.483 to 0.487 to 0.486

#1:LB 0.487 to 0.489 to 0.488

#2:LB 0.489 to 0.489　to 0.489

#3:LB 0.498 to 0.492 to 0.485

#4:LB 0.498 to 0.492 to 0.491

#5:LB 0.497 to 0.504 

#6:LB 0.500 to 0.509 

model_paths = ["../input/roberta-transformers-pytorch/roberta-large/",
                   "../input/roberta-transformers-pytorch/roberta-large/",
                   #"../input/roberta-transformers-pytorch/roberta-large/",
                   #"../input/roberta-transformers-pytorch/distilroberta-base/",
                   #"../input/roberta-transformers-pytorch/roberta-base/",
                   #"../input/roberta-transformers-pytorch/roberta-large/",
                   #"../input/roberta-transformers-pytorch/roberta-base/",
                   #"../input/roberta-transformers-pytorch/roberta-base/"
              ]
weight_paths = ["../input/commonlitreadability-weight/simplelarge_0.469_fold0_mlen250.pt",
                    "../input/commonlitreadability-weight/simplelarge_0.455_fold1_mlen250.pt",
                    #"../input/commonlitreadability-weight/simplelarge_0.464_fold3_mlen250.pt",
                    #"../input/commonlitreadability-weight/bestmodel_0.469_fold1_mlen250.pt",
                    #"../input/commonlitreadability-weight/bestbase_0.443_fold4_mlen250.pt",
                    #"../input/commonlitreadability-weight/multipledropoutrobertalarge_LB0.500_mlen250.pt",
                    #"../input/commonlitreadability-weight/basebestmodel_0.428_fold4_mlen250.pt",
                    #"../input/commonlitreadability-weight/multipledropoutrobertabase_LB0.504_mlen250.pt"
               ]
lengs = [1024,1024]#,1024]#,768,768,1024,768]#,768]


model_paths_emb = [#"../input/roberta-transformers-pytorch/roberta-large/",
                   #"../input/roberta-transformers-pytorch/roberta-large/",
                   "../input/roberta-transformers-pytorch/roberta-large/",
                   "../input/roberta-transformers-pytorch/distilroberta-base/",
                   "../input/roberta-transformers-pytorch/roberta-base/",
                   #"../input/roberta-transformers-pytorch/roberta-large/",]
                   #"../input/roberta-transformers-pytorch/roberta-base/",]
                   #"../input/roberta-transformers-pytorch/roberta-base/"]
                  ]
weight_paths_emb = [#"../input/commonlitreadability-weight/simplelarge_0.469_fold0_mlen250.pt",
                    #"../input/commonlitreadability-weight/simplelarge_0.455_fold1_mlen250.pt",
                    "../input/commonlitreadability-weight/simplelarge_0.464_fold3_mlen250.pt",
                    "../input/commonlitreadability-weight/bestmodel_0.469_fold1_mlen250.pt",
                    "../input/commonlitreadability-weight/bestbase_0.443_fold4_mlen250.pt",
                    #"../input/commonlitreadability-weight/multipledropoutrobertalarge_LB0.500_mlen250.pt",
                    #"../input/commonlitreadability-weight/basebestmodel_0.428_fold4_mlen250.pt",
                    #"../input/commonlitreadability-weight/multipledropoutrobertabase_LB0.504_mlen250.pt"
                   ]
lengs = [1024,768,768]#[1024,1024,1024,768,768,1024,768]#,768]

In [ ]:
#single LB 0.483
preds0 = generate_predictions_md(model_path="../input/roberta-transformers-pytorch/roberta-large/",
                              weight_path="../input/commonlitreadability-weight/simplelarge_0.469_fold0_mlen250.pt",
                              max_len=250,leng=1024)

#single LB 0.487
preds1 = generate_predictions_md(model_path="../input/roberta-transformers-pytorch/roberta-large/",
                              weight_path="../input/commonlitreadability-weight/simplelarge_0.455_fold1_mlen250.pt",
                              max_len=250,leng=1024)

#single LB 0.489
preds2 = generate_predictions_md(model_path="../input/roberta-transformers-pytorch/roberta-large/",
                              weight_path="../input/commonlitreadability-weight/simplelarge_0.464_fold3_mlen250.pt",
                              max_len=250,leng=1024)

In [ ]:
test_embeddings = []

for mpath,wpath, leng in zip(model_paths_emb,weight_paths_emb, lengs):
    emb = generate_embeddings(mpath,wpath, max_len=250,leng=leng)
    test_embeddings.append(emb)

In [ ]:
test_embeddings[0].shape

In [ ]:
import lightgbm as lgb

params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    
    'eta': 0.05,
    'max_depth': 2,
    
    'gamma': 1,
    'subsample': 0.9,
    
    'nthread': 2,
    'tree_method' : 'gpu_hist'
}

params_l = {
    'boosting_type': 'gbdt',
    'metric': 'rmse',
    'objective': 'regression',
    'seed': 2048,
    'learning_rate': 0.05,
    "n_jobs": -1,
    'max_depth': 4,
    "verbose": -1
}

nfolds = 5
kf = KFold(n_splits=nfolds, shuffle=True, random_state=2048)

In [ ]:
def get_pred(train_embeddings,test_emb, targets, train_df):
    best_iterations = []
    oof_rmses = []
    preds = np.zeros(test.shape[0])
    pred_list = []

    for k, (train_idx, valid_idx) in enumerate(kf.split(train_df)):    

        dtrain = xgb.DMatrix(train_embeddings[train_idx], targets[train_idx])
        dvalid = xgb.DMatrix(train_embeddings[valid_idx], targets[valid_idx])
        evals_result = dict()
        booster = xgb.train(params,
                            dtrain,
                            evals=[(dtrain, 'train'), (dvalid, 'valid')],
                            num_boost_round=300,
                            early_stopping_rounds=20,
                            evals_result=evals_result,
                            verbose_eval=False)

        best_iteration = np.argmin(evals_result['valid']['rmse'])
        best_iterations.append(best_iteration)
        oof_rmse = evals_result['valid']['rmse'][best_iteration]
        oof_rmses.append(oof_rmse)
        
        preds += booster.predict(xgb.DMatrix(test_emb), ntree_limit=int(best_iteration+1)) / nfolds
        pred_list.append(booster.predict(xgb.DMatrix(test_emb), ntree_limit=int(best_iteration+1)))
    pred_list = np.array(pred_list)[np.argpartition(oof_rmses, 3)[:2]]
    preds = pred_list.mean(axis=0)
    evals_df = pd.DataFrame()
    evals_df['fold'] = range(1, nfolds+1)
    evals_df['best_iteration'] = best_iterations
    evals_df['oof_rmse'] = oof_rmses

    display(evals_df)
    print('mean oof rmse = {}'.format(np.mean(oof_rmses)))
    
    torch.cuda.empty_cache()
    return preds

def get_pred_lgb(train_embeddings,test_emb, targets, train_df):

    pred = np.zeros(test.shape[0])
    pred_list = []
    rmses = []

    for k, (train_idx, valid_idx) in enumerate(kf.split(train_df)):    

        dtrain = lgb.Dataset(train_embeddings[train_idx], targets[train_idx])
        dvalid = lgb.Dataset(train_embeddings[valid_idx], targets[valid_idx], reference=dtrain)

        model = lgb.train(
            params_l,
            dtrain, 
            num_boost_round=300,
            early_stopping_rounds=20,
            valid_sets=[dtrain, dvalid], 
            verbose_eval=-1
        )

        y_pred = model.predict(train_embeddings[valid_idx])
        rmse = rmse_score(targets[valid_idx], y_pred)
        rmses.append(rmse)

        tmp_pred = model.predict(test_emb)
        pred += tmp_pred / 5
    
    print("\n", "Mean Fold RMSE:", np.mean(rmses))
    return pred

# SVM prediction

In [ ]:
def rmse_score(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))

def get_pred_svm(X,y,X_test,bins,nfolds=5,C=10,kernel='rbf'):
    scores = list()
    preds = np.zeros((X_test.shape[0]))
    
    kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=2048)
    for k, (train_idx,valid_idx) in enumerate(kfold.split(X,bins)):
        model = SVR(C=C,kernel=kernel,gamma='auto')
        X_train,y_train = X[train_idx], y[train_idx]
        X_valid,y_valid = X[valid_idx], y[valid_idx]
        
        model.fit(X_train,y_train)
        prediction = model.predict(X_valid)
        score = rmse_score(prediction,y_valid)
        print(f'Fold {k} , rmse score: {score}')
        scores.append(score)
        preds += model.predict(X_test)
        
    print("mean rmse",np.mean(scores))
    return np.array(preds)/nfolds

In [ ]:
test = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
target = np.load("../input/embedding-featurevector/target.npy")
train_df = pd.read_csv("../input/step1-exclude-anomaly/train.csv")

In [ ]:
num_bins = int(np.floor(1 + np.log2(len(train_df))))
train_df.loc[:,'bins'] = pd.cut(train_df['target'],bins=num_bins,labels=False)
bins = train_df.bins.to_numpy()

In [ ]:
pred_comb_svm = np.empty((0,len(test)))
pred_comb = np.empty((0,len(test)))
pred_comb_lgb = np.empty((0,len(test)))
for i in range(len(test_embeddings)):
    embeddings = np.load("../input/embedding-featurevector/embeddings" + str(i+2) + ".npy")
    print(embeddings.shape)
    preds = get_pred(embeddings,test_embeddings[i], target, train_df)
    preds_lgb = get_pred_lgb(embeddings,test_embeddings[i], target, train_df)
    pred_comb = np.append(pred_comb,preds[None,:],axis=0)
    pred_comb_lgb = np.append(pred_comb_lgb,preds_lgb[None,:],axis=0)
    #preds_svm = get_pred_svm(embeddings, target, test_embeddings[i], bins = bins)
    #pred_comb_svm = np.append(pred_comb_svm,preds_svm[None,:],axis=0)

In [ ]:
pred_ori = (preds0 + preds1 + preds2)/3
#pred_ori = (preds0 + preds1)/2
pred_svm = pred_comb_svm.sum(axis=0)/len(pred_comb_svm)
pred_xgb = pred_comb.sum(axis=0)/len(pred_comb)
pred_lgb = pred_comb_lgb.sum(axis=0)/len(pred_comb_lgb)
predf = (pred_ori*3 + pred_xgb*3)/6 # +pred_comb_svm[1])/6

# model2

In [ ]:
BATCH_SIZE = 32
MAX_LEN = 248
EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
ROBERTA_PATH = "../input/roberta-transformers-pytorch/roberta-base"
TOKENIZER_PATH = "../input/roberta-transformers-pytorch/roberta-base"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

test_df = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")
submission_df = pd.read_csv("/kaggle/input/commonlitreadabilityprize/sample_submission.csv")
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

In [ ]:
class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7})                       
        
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config)  
            
        self.attention = nn.Sequential(            
            nn.Linear(768, 512),            
            nn.Tanh(),                       
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(768, 1)                        
        )
        

    def forward(self, input_ids, attention_mask):
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]

        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(context_vector)

In [ ]:
class LitDataset(Dataset):
    def __init__(self, df, inference_only=False):
        super().__init__()

        self.df = df        
        self.inference_only = inference_only
        self.text = df.excerpt.tolist()
        #self.text = [text.replace("\n", " ") for text in self.text]
        
        if not self.inference_only:
            self.target = torch.tensor(df.target.values, dtype=torch.float32)        
    
        self.encoded = tokenizer.batch_encode_plus(
            self.text,
            padding = 'max_length',            
            max_length = MAX_LEN,
            truncation = True,
            return_attention_mask=True
        )        
 

    def __len__(self):
        return len(self.df)

    
    def __getitem__(self, index):        
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        
        if self.inference_only:
            return (input_ids, attention_mask)            
        else:
            target = self.target[index]
            return (input_ids, attention_mask, target)

In [ ]:
def predict(model, data_loader):
    """Returns an np.array with predictions of the |model| on |data_loader|"""
    model.eval()

    result = np.zeros(len(data_loader.dataset))    
    index = 0
    
    with torch.no_grad():
        for batch_num, (input_ids, attention_mask) in enumerate(data_loader):
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)
                        
            pred = model(input_ids, attention_mask)                        

            result[index : index + pred.shape[0]] = pred.flatten().to("cpu")
            index += pred.shape[0]

    return result

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoConfig
import gc
gc.enable()

test_dataset = LitDataset(test_df, inference_only=True)

NUM_MODELS = 5

all_predictions = np.zeros((NUM_MODELS, len(test_df)))



test_dataset = LitDataset(test_df, inference_only=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         drop_last=False, shuffle=False, num_workers=2)

for model_index in range(NUM_MODELS):            
    model_path = f"../input/commonlit-roberta-0467/model_{model_index + 1}.pth"
    print(f"\nUsing {model_path}")
                        
    model = LitModel()
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))    
    model.to(DEVICE)
        
    all_predictions[model_index] = predict(model, test_loader)
            
    del model
    gc.collect()

In [ ]:
model_predictions = (all_predictions[0] + all_predictions[2] + all_predictions[3])/3
model_predictions5 = all_predictions.mean(axis=0)

In [ ]:
pred_2sigma = (pred_ori*3 + model_predictions5*5)/8

In [ ]:
#predfinal = (predf + model_predictions)/2

In [ ]:
predfinal_x = (pred_ori*3 + pred_xgb*2 + model_predictions*5)/10
predfinal_l = (pred_ori*3 + pred_lgb*2 + model_predictions*5)/10

In [ ]:
train_ex = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
train_ex = train_ex[train_ex.target!=0]
upper = train_ex.target.mean()+1.0*train_ex.target.std()
lower = train_ex.target.mean()-1.0*train_ex.target.std()

#plt.scatter(train_ex.target.values,train_ex.standard_error.values)
#print(lower,upper)

In [ ]:
test["best_pred"] = (pred_ori*3 + model_predictions*3)/6
test['pred2sigma'] =pred_2sigma
test['predfull'] = predfinal_l
test = test.sort_values("best_pred")
sigma2 = int(len(test)*0.05)
headindex = test.head(sigma2).index
tailindex = test.tail(sigma2).index
bestpredindex = test[(test.best_pred > lower) & (test.best_pred < upper)].index
fullindex = test[~((test.best_pred > lower) & (test.best_pred < upper))].index
meansindex = test[(test.best_pred > train_ex.target.max()) | (test.best_pred < train_ex.target.min())].index
test["prediction"] = 0
test.loc[fullindex,"prediction"] = test.loc[fullindex,"predfull"]
test.loc[bestpredindex,"prediction"] = test.loc[bestpredindex,"best_pred"]
test.loc[meansindex,"prediction"] = train_ex.target.mean()
test.loc[headindex,"prediction"] = test.loc[headindex,'pred2sigma']
test.loc[tailindex,"prediction"] = test.loc[tailindex,'pred2sigma']
test

In [ ]:
submission = pd.DataFrame()
submission['id'] = test['id'].copy()
submission['target'] = test['prediction'].copy()
submission.to_csv('submission.csv', index=False)
submission